In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [12]:
import os
import numpy as np
import torch
from pathlib import Path

print(torch.__version__)

1.9.0+cu111


In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0" to  "7"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [5]:
torch.cuda.empty_cache()

In [7]:
import sys

# if os.path.join(os.getcwd(), "detr/") not in sys.path:
#     sys.path.append(os.path.join(os.getcwd(), "detr/"))
    
if os.path.join(os.getcwd(), "detr-fine/") not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), "detr-fine/"))

# print(sys.path)

In [ ]:
from main import args
import torchvision.transforms as T
from model.detr import build_model
from PIL import Image
import matplotlib.pyplot as plt
import math

out_dir = Path('./out_dir')
impath = Path('./data/images/aac_blocks_2176.jpg')

checkpoint = torch.load(out_dir / "checkpoint.pth")

model, criterion, postprocessors = build_model(args)
model.load_state_dict(checkpoint['model'])
model.eval();

transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

im = Image.open(impath)
img = transform(im).unsqueeze(0)
out = model(img)

result = postprocessors['panoptic'](out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]

scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]
# threshold the confidence
keep = scores > 0.1

keep

# Plot all the remaining masks
ncols = 2
fig, axs = plt.subplots(ncols=ncols, nrows=math.ceil(keep.sum().item() / ncols), figsize=(18, 10))
for line in axs:
    for a in line:
        a.axis('off')
for i, mask in enumerate(out["pred_masks"][keep]):
    ax = axs[i // ncols, i % ncols]
    ax.imshow(mask.detach().numpy(), cmap="cividis")
    ax.axis('off')
fig.tight_layout()